## Library Import

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import psycopg2
from psycopg2 import Error
import os
from dotenv import load_dotenv
import datetime as dt

## SQL Connection Set UP

In [2]:
# Read in ENV credentials 

load_dotenv()

username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')

In [3]:
# Connection to db 
USER = username
PSWD = password
HOST = "pg.analytics.northwestern.edu"
PORT = "5432"
DB_NAME = "everything2023"

# Connect to postgress database
try:
    # Connect to an existing database
    connection = psycopg2.connect(user = USER,
                                  password = PSWD,
                                  host = HOST,
                                  port = PORT,
                                  database = DB_NAME)
    cursor = connection.cursor()
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")
    
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

You are connected to -  ('PostgreSQL 10.12 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-39), 64-bit',) 



## FEATURE SELECTION

In [4]:
# Read trnsact table
sql_cmd = """
WITH DailyReturnRates AS (
    SELECT 
        "SALEDATE", 
        "STORE", 
        SUM(CASE WHEN "STYPE" = 'R' THEN 1.0 ELSE 0.0 END) / 
        (SUM(CASE WHEN "STYPE" = 'P' THEN 1.0 ELSE 0.0 END) + SUM(CASE WHEN "STYPE" = 'R' THEN 1.0 ELSE 0.0 END)) AS "ReturnRate",
        COUNT(*) AS "NumTrans",
        SUM(CASE WHEN "SPRICE" < "ORGPRICE" THEN 1 ELSE 0 END) AS "DiscountSalesCount",
        SUM("AMT") AS "AmtTrans",
        AVG("SPRICE") AS "AvgPrice"
    FROM 
        group_14.trnsact
    GROUP BY 
        "SALEDATE", 
        "STORE"
)
SELECT 
    "SALEDATE", 
    "STORE", 
    "ReturnRate",
    LAG("ReturnRate", 1) OVER (PARTITION BY "STORE" ORDER BY "SALEDATE") AS "PreviousDayReturnRate",
    LAG("ReturnRate", 7) OVER (PARTITION BY "STORE" ORDER BY "SALEDATE") AS "PreviousWeekReturnRate",
    LAG("ReturnRate", 30) OVER (PARTITION BY "STORE" ORDER BY "SALEDATE") AS "PreviousMonthReturnRate",
    "NumTrans",
    "DiscountSalesCount",
    "AmtTrans",
    "AvgPrice", 
    LAG(drr."NumTrans", 1) OVER (PARTITION BY drr."STORE" ORDER BY drr."SALEDATE") AS "PreviousDayTrans",
    LAG(drr."AmtTrans", 1) OVER (PARTITION BY drr."STORE" ORDER BY drr."SALEDATE") AS "PreviousDayAmtTrans",
    (SELECT SUM(subdrr."NumTrans")
     FROM DailyReturnRates subdrr
     WHERE subdrr."STORE" = drr."STORE" 
     AND subdrr."SALEDATE" BETWEEN drr."SALEDATE" - INTERVAL '30 days' AND drr."SALEDATE") AS "Last30DaysTrans",
    (SELECT SUM(subdrr."AmtTrans")
     FROM DailyReturnRates subdrr
     WHERE subdrr."STORE" = drr."STORE" 
     AND subdrr."SALEDATE" BETWEEN drr."SALEDATE" - INTERVAL '30 days' AND drr."SALEDATE") AS "Last30DaysAmtTrans"
FROM 
    DailyReturnRates drr
ORDER BY 
    "SALEDATE", 
    "STORE";
"""
return_df = pd.read_sql(sql_cmd, connection);

/var/folders/x7/1lss9g1950lgrxkwhp8c6c180000gn/T/ipykernel_59259/2419310726.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return_df = pd.read_sql(sql_cmd, connection);


In [1]:
## Output our df so we do not need to run SQL everytime

In [5]:
return_df.to_csv("return_df.csv", index = False)

## ADD location feature

In [9]:
sql_cmd = """
SELECT "STATE", "STORE"
FROM 
  group_14.strinfo

"""
strinfo = pd.read_sql(sql_cmd, connection);
strinfo.head()

/var/folders/x7/1lss9g1950lgrxkwhp8c6c180000gn/T/ipykernel_42175/3992140737.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  strinfo = pd.read_sql(sql_cmd, connection);


STATE  STORE
0    FL      2
1    MO      3
2    AR      4
3    TX      7
4    AZ      9

In [14]:
new_return_df = return_df.merge(strinfo, how = "left", on = "STORE")

In [15]:
new_return_df.to_csv("return_df_with_store.csv", index = False)

In [16]:
# Extract different time features 
return_rate_df['Month'] = pd.to_datetime(return_rate_df['SALEDATE']).dt.month
return_rate_df['Quarter'] = pd.to_datetime(return_rate_df['SALEDATE']).dt.quarter
return_rate_df['Day'] = pd.to_datetime(return_rate_df['SALEDATE']).dt.dayofweek
return_rate_df

SALEDATE  STORE  ReturnRate  NumTrans  DiscountSalesCount  AmtTrans  \
0       2004-08-01    102    0.046968      1171                 533  34507.81   
1       2004-08-01    103    0.059271       658                 349  16873.44   
2       2004-08-01    107    0.062332      1492                 716  34605.01   
3       2004-08-01    202    0.054707       786                 420  17108.43   
4       2004-08-01    203    0.100752       665                 423  16585.97   
...            ...    ...         ...       ...                 ...       ...   
125915  2005-08-27   9704    0.064167      1200                 911  20079.25   
125916  2005-08-27   9709    0.051787      1371                 952  24007.62   
125917  2005-08-27   9804    0.056711      1587                1173  29260.54   
125918  2005-08-27   9806    0.002625      1143                 516  33762.85   
125919  2005-08-27   9909    0.045962      1523                1174  24834.76   

        Month  Quarter  Day  
0           8        3    6  
1           8        3    6  
2           8        3    6  
3           8        3    6  
4           8        3    6  
...       ...      ...  ...  
125915      8        3    5  
125916      8        3    5  
125917      8        3    5  
125918      8        3    5  
125919      8        3    5  

[125920 rows x 9 columns]